In [1]:
input_file = '/content/prepared_data.csv'
output_file = '/content/cleaned_data.csv'

# Count rows from 0, so row 190149 is line 190150
row_to_remove = 190149

with open(input_file, 'r', encoding='utf-8') as infile, \
     open(output_file, 'w', encoding='utf-8') as outfile:

    for i, line in enumerate(infile):
        if i != row_to_remove:
            outfile.write(line)

print(f"Removed row {row_to_remove}, saved to {output_file}")

Removed row 190149, saved to /content/cleaned_data.csv


In [2]:
import pandas as pd
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch

# Try using the CSV module directly for more control
import csv

data = []
with open('/content/cleaned_data.csv', 'r', encoding='utf-8') as f:
    reader = csv.reader(f)
    for i, row in enumerate(reader):
        if i == 2938:  # Skip or fix the problematic row
            continue  # or manually fix this row
        data.append(row)

df = pd.DataFrame(data[1:], columns=data[0])  # Assuming first row is header

# Convert to dictionary for easier processing
data = df.to_dict('records')

In [3]:
# Load the pretrained T5-small model and tokenizer
model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Set device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [4]:
from transformers import Trainer, TrainingArguments
from torch.utils.data import Dataset

# Create a custom dataset class
class SummaryDataset(Dataset):
    def __init__(self, data, tokenizer, max_input_len=512, max_output_len=150):
        self.data = data
        self.tokenizer = tokenizer
        self.max_input_len = max_input_len
        self.max_output_len = max_output_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        item = self.data[index]

        # Combine title and content for better context
        input_text = f"summarize: {item['title']}. {item['content']}"

        # Tokenize inputs
        inputs = self.tokenizer(
            input_text,
            max_length=self.max_input_len,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )

        # Tokenize targets (using content as target for self-supervised learning)
        # In a real scenario, you'd have human-written summaries as targets
        targets = self.tokenizer(
            item['content'][:500],  # Using first 500 chars as pseudo-summary
            max_length=self.max_output_len,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )

        return {
            "input_ids": inputs["input_ids"].flatten(),
            "attention_mask": inputs["attention_mask"].flatten(),
            "labels": targets["input_ids"].flatten()
        }

# Create dataset
train_dataset = SummaryDataset(data, tokenizer)

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=8,
    save_steps=10_000,
    save_total_limit=2,
    learning_rate=3e-5,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=100,
    gradient_accumulation_steps=2,  # Simulate larger batch size
    max_steps=10,                 # Hard limit on steps
    fp16=True,
)

# Create trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)

# Start training
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: prodesignamcontact (prodesignamcontact-pro-design-am) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss


TrainOutput(global_step=10, training_loss=4.638725280761719, metrics={'train_runtime': 513.0272, 'train_samples_per_second': 0.312, 'train_steps_per_second': 0.019, 'total_flos': 21654688235520.0, 'train_loss': 4.638725280761719, 'epoch': 0.0008560178051703475})

In [5]:
def generate_summary(text, model, tokenizer):
    # Prepare input
    input_text = f"summarize: {text}"
    inputs = tokenizer.encode(
        input_text,
        return_tensors="pt",
        max_length=512,
        truncation=True
    ).to(device)

    # Generate summary
    outputs = model.generate(
        inputs,
        max_length=150,
        min_length=40,
        length_penalty=2.0,
        num_beams=4,
        early_stopping=True
    )

    # Decode and return
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Sample 200 articles (adjust if needed)
sample_df = df.sample(n=200, random_state=42).reset_index(drop=True)

# Apply to your dataframe
sample_df['t5_summary'] = sample_df['content'].apply(lambda x: generate_summary(str(x), model, tokenizer))

# Save results
df.to_csv('t5_summarized_data.csv', index=False)

In [8]:
from rouge_score import rouge_scorer

def evaluate_rouge(references, hypotheses):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = []

    for ref, hyp in zip(references, hypotheses):
        scores.append(scorer.score(ref, hyp))

    return {
        'rouge1': sum(s['rouge1'].fmeasure for s in scores) / len(scores),
        'rouge2': sum(s['rouge2'].fmeasure for s in scores) / len(scores),
        'rougeL': sum(s['rougeL'].fmeasure for s in scores) / len(scores),
    }

# Evaluate
rouge_scores = evaluate_rouge(sample_df['content'], sample_df['t5_summary'])
print(rouge_scores)

{'rouge1': 0.18006368350220509, 'rouge2': 0.15699425337564152, 'rougeL': 0.16703938504106522}
